In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Cox-Ross-Rubinstein Binomial Tree for American Call

In [2]:
def AmCall(S0, K, r, dyield, sigma, T, N=2000):
    dT = float(T) / N
    
    # up and down factor will be constant for the tree so we calculate outside the loop
    u = np.exp(sigma * np.sqrt(dT))
    d = 1.0 / u
 
    #investor holds option, initialization of vector H 
    H =  np.asarray([0.0 for i in range(N + 1)])
        
    #we need the stock tree for calculations of expiration values
    S = np.asarray([(S0 * u**j * d**(N - j)) for j in range(N + 1)])
    
    #we vectorize the strikes as well so the expiration check will be faster
    K = np.asarray( [float(K) for i in range(N + 1)])
    
 
    #rates are fixed so the probability of up and down are fixed.
    #this is used to make sure the drift is the risk free rate
    
    #note that dividend yield appears in computing p but not in discounting
    
    a = np.exp((r-dyield) * dT)
    p = (a - d)/ (u - d)
 
    # Compute the leaves
    H[:] = np.maximum(S-K, 0.0)
   
    #calculate backward the option prices
    for i in range(N-1, -1, -1):
        H[:-1]=np.exp(-r * dT) * (p * H[1:] + (1-p) * H[:-1])
        S[:]=S[:]*u
        
        H[:]=np.maximum(H[:],S[:]-K[:])
    return H[0]

In [3]:
AmCall(47, 47, 0.08, 0.08, 0.2, 1, N=20000)

3.5254447719914137

## 2. Convertible Bond model Goldman 1994 (by E. Derman)

In [4]:
def round_down(num, divisor):
    return num - (num%divisor)

In [5]:
def Goldman(F=100, c={1: 0.1}, T=5.0, ratio=1.0, calls={2:115, 3:110, 4:105, 5:100}, puts={3: 120}, 
            S0=100, dyield=0.0, sigma=0.1, r=0.05, mu=0.05, dd=500, N=200):
    dT = float(T) / N

    # Coupons
    freq = list(c.keys())[0]
    Tc   = 1/freq
    
    # up and down factor will be constant for the tree so we calculate outside the loop
    u = np.exp((mu-dyield-0.5*sigma**2)*dT+sigma*np.sqrt(dT))
    d = np.exp((mu-dyield-0.5*sigma**2)*dT-sigma*np.sqrt(dT))

    #investor holds convertible, initialization of vector H, V, p and y
    H = np.asarray([0.0 for i in range(N + 1)])
    V = np.asarray([0.0 for i in range(N + 1)])
    p = np.asarray([0.0 for i in range(N + 1)])
    y = np.asarray([0.0 for i in range(N + 1)])
    
    #we need the stock tree for calculations of expiration values
    S = np.asarray([(S0 * u**j * d**(N - j)) for j in range(N + 1)])
        
    # Compute the leaves
    B = F+F*c[freq]*(T-round_down(T-1e-5, Tc))
    V = np.maximum(ratio*S, B)
    p[V>B] = 1.0
    y = p*r + (1-p)*(r+dd/1e4)
    
    #calculate backward the tree the convertible bond value
    for i in range(N-1, -1, -1):
        t = i*dT
        p = 0.5*(p[1:]+p[:-1])   # = 0.5*(p_u + p_d)
        H = 0.5*(np.exp(-y[1:]*dT)*V[1:] + np.exp(-y[:-1]*dT)*V[:-1])
        S = S[:-1] * u/np.exp((mu-dyield-0.5*sigma**2)*dT)
        
        # Put, Call, readjustment of probabilities and discount factors 
        a = F*c[freq]*(t-round_down(t-1e-5, Tc))        
        
        for key, value in calls.items():
            if key == round_down(t-1e-5,1):
                C = value + a
            else:
                C = 1e5
        
        for key, value in puts.items():
            if key == round_down(t-1e-5,1):
                P = value + a
            else:
                P = 0.0
        
        temp = np.minimum(H, C)
        temp = np.maximum(temp, P)
        V    = np.maximum(temp, ratio*S)
        p[V==C] = 0.0
        p[V==P] = 0.0

        y = p*r + (1-p)*(r+dd/1e4)

    return V[0]

In [6]:
Goldman()

125.23227161918943

In [73]:
Goldman(F=100, c={1: 0.05}, T=1.95, ratio=1000.0, calls={1:1e5}, puts={2: 0.0}, 
            S0=0.075, dyield=0.0, sigma=0.45, r=0.0188, mu=0.123062, dd=3312, N=2000)

81.413603667922303

## 3. Animated plots of the behaviour of Convertible Bonds 

In [3]:
li=[[1,[[2]],[[[3]]]],[['4'],{5:5}]]
flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
print(flatten(li))

[1, 2, 3, '4', {5: 5}]
